# Machine Learning & Synthetic Data

For this notebook, I'll use the adjusted set from feature engineering notebook on Logit, SVM and Neural Network models.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn import metrics
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [2]:
# Import datasets
train = pd.read_csv('train_adj.csv')
test = pd.read_csv('test_adj.csv')